In [3]:
import tensorflow as tf 
import random 

ENC_SIZE = 4
ENC_LAYERSIZE = 50
DEC_LAYERSIZE = 50
LEARN_LAYERSIZE = 50
RANGE = 10

def func(x,y): return x*y

def encode_float(input_tensor):
    with tf.variable_scope("encode", reuse=tf.AUTO_REUSE):
        iw_1 = tf.get_variable("iw_1", shape=[1,ENC_LAYERSIZE])
        ib_1 = tf.get_variable("ib_1", shape=[ENC_LAYERSIZE,])
        l1 = tf.nn.relu(tf.add(tf.matmul(input_tensor, iw_1), ib_1))
        
        iw_2 = tf.get_variable("iw_2", shape=[ENC_LAYERSIZE,ENC_LAYERSIZE/2])
        ib_2 = tf.get_variable("ib_2", shape=[ENC_LAYERSIZE/2,])
        l2 = tf.nn.relu(tf.add(tf.matmul(l1, iw_2), ib_2))
        
        iw_3 = tf.get_variable("iw_3", shape=[ENC_LAYERSIZE/2,ENC_SIZE])
        ib_3 = tf.get_variable("ib_3", shape=[ENC_SIZE,])
        l3 = tf.nn.relu(tf.add(tf.matmul(l2, iw_3), ib_3))
    return l3

def decode_float(input_encoded):
    with tf.variable_scope("decode", reuse=tf.AUTO_REUSE):
        iw_1 = tf.get_variable("iw_1", shape=[ENC_SIZE,DEC_LAYERSIZE])
        ib_1 = tf.get_variable("ib_1", shape=[DEC_LAYERSIZE,])
        l1 = tf.nn.relu(tf.add(tf.matmul(input_encoded, iw_1), ib_1))
        
        iw_2 = tf.get_variable("iw_2", shape=[DEC_LAYERSIZE,DEC_LAYERSIZE/2])
        ib_2 = tf.get_variable("ib_2", shape=[DEC_LAYERSIZE/2,])
        l2 = tf.nn.relu(tf.add(tf.matmul(l1, iw_2), ib_2))
        
        iw_3 = tf.get_variable("iw_3", shape=[DEC_LAYERSIZE/2,1])
        ib_3 = tf.get_variable("ib_3", shape=[1,])
        l3 = tf.add(tf.matmul(l2, iw_3), ib_3)
    return l3 

def learn_func(x,y):
    x_enc = encode_float(x)
    y_enc = encode_float(y)
    expected_enc = encode_float(expected)
    inputs = tf.concat([x_enc,y_enc],1)
    with tf.variable_scope("learn", reuse=tf.AUTO_REUSE):
        iw_1 = tf.get_variable("iw_1", shape=(2*ENC_SIZE, LEARN_LAYERSIZE))
        ib_1 = tf.get_variable("ib_1", shape=(LEARN_LAYERSIZE,))
        l1 = tf.nn.relu(tf.add(tf.matmul(inputs, iw_1), ib_1))
        
        iw_2 = tf.get_variable("iw_2", shape=(LEARN_LAYERSIZE, LEARN_LAYERSIZE/2))
        ib_2 = tf.get_variable("ib_2", shape=(LEARN_LAYERSIZE/2,))
        l2 = tf.nn.relu(tf.add(tf.matmul(l1, iw_2), ib_2))
        
        iw_3 = tf.get_variable("iw_3", shape=(LEARN_LAYERSIZE/2, ENC_SIZE))
        ib_3 = tf.get_variable("ib_3", shape=(ENC_SIZE,))
        l3 = tf.nn.relu(tf.add(tf.matmul(l2, iw_3), ib_3))
    return l3 


x = tf.placeholder(shape=(None,1), dtype=tf.float32, name='x')
y = tf.placeholder(shape=(None,1), dtype=tf.float32, name='y')
expected = tf.placeholder(shape=(None,1), dtype=tf.float32, name='expected')

learned = learn_func(x,y)
# loss = tf.reduce_mean(tf.losses.mean_squared_error(learned, encode_float(expected)) ) + tf.reduce_mean(tf.losses.mean_squared_error(expected, decode_float(learned)))
loss = (tf.losses.mean_squared_error(expected, decode_float(learned)) + 
        tf.losses.mean_squared_error(encode_float(expected), learned) + 
        tf.losses.mean_squared_error(x, decode_float(encode_float(x)))  + 
        tf.losses.mean_squared_error(y, decode_float(encode_float(y))) +
        tf.losses.mean_squared_error(expected, decode_float(encode_float(expected))))

with tf.Session() as sess: 
    with tf.variable_scope("sess", reuse=tf.AUTO_REUSE):
        optimizer = tf.train.AdamOptimizer().minimize(loss)
    sess.run([tf.local_variables_initializer(), 
              tf.global_variables_initializer()])
    
    print "TRAIN"
    train_iterations = 25000
    batch_size = 100 
    for i in xrange(train_iterations):
        feed = { x:[], y:[], expected:[]}
        for b in range(batch_size):
            ix = random.uniform(-RANGE, RANGE)
            iy = random.uniform(-RANGE, RANGE)
            e = func(ix, iy)
            feed[x].append([ix,])
            feed[y].append([iy,])
            feed[expected].append([e,])
        _, l = sess.run([optimizer, loss], feed_dict=feed)
        if i % 2500 == 0:
            print "i=",i," loss=", l
    for i in range(-RANGE, RANGE):
        f = float(i)
        enc = sess.run([encode_float([[f,],])])
        print "enc = ", f, " = ", enc
        print "dec = ", sess.run([decode_float(enc[0])])
    for i in range(25):
        tx = random.uniform(-RANGE,RANGE)
        ty = random.uniform(-RANGE,RANGE) 
        te = func(tx, ty) 
        print tx, ty, te 
#        print "learn = ", sess.run([learn_func([[tx,],],[[ty,],])], feed_dict={x:[[tx,],], y:[[ty,],], expected:[[te,],]})
        print "decode_learn = ", sess.run([decode_float(learn_func([[tx,],],[[ty,],]))], feed_dict={x:[[tx,],], y:[[ty,],], expected:[[te,],]})
        print "loss = ", sess.run([loss], feed_dict={x:[[tx,],], y:[[ty,],], expected:[[te,],]})


TRAIN
i= 0  loss= 2017.81
i= 2500  loss= 0.960452
i= 5000  loss= 0.52289
i= 7500  loss= 0.28749
i= 10000  loss= 0.121323
i= 12500  loss= 0.161903
i= 15000  loss= 0.271373
i= 17500  loss= 0.103738
i= 20000  loss= 0.164337
i= 22500  loss= 0.11577
enc =  -10.0  =  [array([[ 2.34517288,  0.27773932,  0.        ,  0.        ]], dtype=float32)]
dec =  [array([[-9.99434853]], dtype=float32)]
enc =  -9.0  =  [array([[ 2.08889246,  0.11650184,  0.        ,  0.        ]], dtype=float32)]
dec =  [array([[-8.9802351]], dtype=float32)]
enc =  -8.0  =  [array([[ 1.82675648,  0.        ,  0.        ,  0.0213536 ]], dtype=float32)]
dec =  [array([[-7.99868107]], dtype=float32)]
enc =  -7.0  =  [array([[ 1.5655961 ,  0.        ,  0.        ,  0.12094355]], dtype=float32)]
dec =  [array([[-6.97428942]], dtype=float32)]
enc =  -6.0  =  [array([[ 1.3299427 ,  0.        ,  0.        ,  0.18070233]], dtype=float32)]
dec =  [array([[-5.96759796]], dtype=float32)]
enc =  -5.0  =  [array([[ 1.10779285,  0.    